In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

from src.logger import logging
from catboost import CatBoostRegressor
from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
import optuna
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import os
from sklearn.model_selection import cross_val_score
import sys

In [2]:
print(os.getcwd())

d:\DS_project\ML_project\src\components


In [ ]:
print('test')

In [28]:
sys.path.append('D:\DS_project\ML_project\src\components')

In [25]:
sys.path

['d:\\DS_project\\ML_project\\src\\components',
 'd:\\DS_project\\ML_project\\venv\\python38.zip',
 'd:\\DS_project\\ML_project\\venv\\DLLs',
 'd:\\DS_project\\ML_project\\venv\\lib',
 'd:\\DS_project\\ML_project\\venv',
 '',
 'd:\\DS_project\\ML_project\\venv\\lib\\site-packages',
 'd:\\DS_project\\ML_project\\venv\\lib\\site-packages\\ml_project-0.0.1-py3.8.egg',
 'd:\\DS_project\\ML_project\\venv\\lib\\site-packages\\win32',
 'd:\\DS_project\\ML_project\\venv\\lib\\site-packages\\win32\\lib',
 'd:\\DS_project\\ML_project\\venv\\lib\\site-packages\\Pythonwin',
 'D:\\DS_project\\ML_project\\src',
 'D:\\DS_project\\ML_project\\src\\**',
 'D:\\DS_project\\ML_project',
 'D:\\DS_project\\ML_project\\src\\components',
 'D:\\DS_project\\ML_project\\src',
 'D:\\DS_project\\ML_project\\src']

In [27]:
from exception import CustomException

In [34]:
from data_tranformation import DataTransformation

ImportError: cannot import name 'CustomException' from 'src.exception' (d:\DS_project\ML_project\venv\lib\site-packages\ml_project-0.0.1-py3.8.egg\src\exception.py)

In [15]:
from src.logger import logging

In [85]:
from sklearn.metrics import mean_squared_error

class ModelTrainer:
    def initiate_model_train(self, X_train, X_test, y_train, y_test):
        """
        Choose models and perform hyperparameter with optuna
        """
        def objective(trial):
            model_name = trial.suggest_categorical("model_name", ["Random Forest", "Decision Tree", "Gradient Boosting", "Linear Regression", "XGBRegressor", "CatBoosting Regressor", "AdaBoost Regressor"])

            if model_name == "Random Forest":
                model = RandomForestRegressor(
                    n_estimators=trial.suggest_int("rf_n_estimators", 8, 256),
                    criterion='mse'  # Change 'mse' to 'mae' if you want to use Mean Absolute Error
                )
            elif model_name == "Decision Tree":
                model = DecisionTreeRegressor(
                    criterion='mse',
                    splitter=trial.suggest_categorical("dt_splitter", ['best', 'random'])
                )
            elif model_name == "Gradient Boosting":
                model = GradientBoostingRegressor(
                    learning_rate=trial.suggest_float("gb_learning_rate", 0.001, 0.1),
                    n_estimators=trial.suggest_int("gb_n_estimators", 8, 256),
                    subsample=trial.suggest_float("gb_subsample", 0.6, 0.9),
                    criterion='friedman_mse'  # Change 'friedman_mse' to 'squared_error' if needed
                )
            elif model_name == "Linear Regression":
                model = LinearRegression()
            elif model_name == "XGBRegressor":
                model = XGBRegressor(
                    learning_rate=trial.suggest_float("xgb_learning_rate", 0.001, 0.1),
                    n_estimators=trial.suggest_int("xgb_n_estimators", 8, 256)
                )
            elif model_name == "CatBoosting Regressor":
                model = CatBoostRegressor(
                    depth=trial.suggest_int("cb_depth", 6, 10),
                    learning_rate=trial.suggest_float("cb_learning_rate", 0.01, 0.1),
                    iterations=trial.suggest_int("cb_iterations", 30, 100),
                    verbose=False
                )
            elif model_name == "AdaBoost Regressor":
                model = AdaBoostRegressor(
                    learning_rate=trial.suggest_float("ab_learning_rate", 0.001, 0.1),
                    n_estimators=trial.suggest_int("ab_n_estimators", 8, 256)
                )

            # Perform cross-validation to evaluate the model
            scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')  # Use MSE as the scoring metric
            score = -scores.mean()  # Take the negative mean as cross_val_score returns negative MSE

            return score

        # Set up the Optuna study and optimize the objective function
        study = optuna.create_study(direction="minimize")  # minimize MSE
        study.optimize(objective, n_trials=20)

        # Instantiate the best model based on the Optuna results
        best_model_name = study.best_trial.params["model_name"]
        if best_model_name == "Random Forest":
            best_model = RandomForestRegressor(**study.best_trial.params)
        elif best_model_name == "Decision Tree":
            best_model = DecisionTreeRegressor(**study.best_trial.params)
        elif best_model_name == "Gradient Boosting":
            best_model = GradientBoostingRegressor(**study.best_trial.params)
        elif best_model_name == "Linear Regression":
            best_model = LinearRegression()
        elif best_model_name == "XGBRegressor":
            best_model = XGBRegressor(**study.best_trial.params)
        elif best_model_name == "CatBoosting Regressor":
            best_model = CatBoostRegressor(**study.best_trial.params, verbose=False)
        elif best_model_name == "AdaBoost Regressor":
            best_model = AdaBoostRegressor(**study.best_trial.params)

        # Train the best model on the entire training set
        best_model.fit(X_train, y_train)
        logging.info('Find best model completed')
        return best_model


In [91]:
from sklearn.metrics import mean_squared_error

class ModelTrainer:
    def initiate_model_train(self, X_train, X_test, y_train, y_test):
        """
        Choose models and perform hyperparameter with optuna
        """
        def objective(trial):
            model_name = trial.suggest_categorical("model_name", ["Random Forest", "Decision Tree", "Gradient Boosting", "Linear Regression", "XGBRegressor", "CatBoosting Regressor", "AdaBoost Regressor"])
            if model_name == "Random Forest":
                model = RandomForestRegressor(
                    n_estimators=trial.suggest_int("rf_n_estimators", 8, 256),
                    criterion='squared_error'  # Modify as needed
                )
            elif model_name == "Decision Tree":
                model = DecisionTreeRegressor(
                    criterion='squared_error',  # Change to one of the allowed options
                    splitter=trial.suggest_categorical("dt_splitter", ['best', 'random'])
                )
            elif model_name == "Gradient Boosting":
                model = GradientBoostingRegressor(
                    learning_rate=trial.suggest_float("gb_learning_rate", 0.001, 0.1),
                    n_estimators=trial.suggest_int("gb_n_estimators", 8, 256),
                    subsample=trial.suggest_float("gb_subsample", 0.6, 0.9),
                    criterion='friedman_mse'  # Modify as needed
                )
            elif model_name == "Linear Regression":
                model = LinearRegression()
            elif model_name == "XGBRegressor":
                model = XGBRegressor(
                    learning_rate=trial.suggest_float("xgb_learning_rate", 0.001, 0.1),
                    n_estimators=trial.suggest_int("xgb_n_estimators", 8, 256)
                )
            elif model_name == "CatBoosting Regressor":
                model = CatBoostRegressor(
                    depth=trial.suggest_int("cb_depth", 6, 10),
                    learning_rate=trial.suggest_float("cb_learning_rate", 0.01, 0.1),
                    iterations=trial.suggest_int("cb_iterations", 30, 100),
                    verbose=False
                )
            elif model_name == "AdaBoost Regressor":
                model = AdaBoostRegressor(
                    learning_rate=trial.suggest_float("ab_learning_rate", 0.001, 0.1),
                    n_estimators=trial.suggest_int("ab_n_estimators", 8, 256)
                )

            # Perform cross-validation to evaluate the model
            scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')  # Use MSE as the scoring metric
            score = -scores.mean()  # Take the negative mean as cross_val_score returns negative MSE

            return score

        # Set up the Optuna study and optimize the objective function
        study = optuna.create_study(direction="minimize")  # minimize MSE
        study.optimize(objective, n_trials=20)

        # Instantiate the best model based on the Optuna results
        best_model_name = study.best_trial.params["model_name"]
        if best_model_name == "Random Forest":
            best_model = RandomForestRegressor(**study.best_trial.params)
        elif best_model_name == "Decision Tree":
            best_model = DecisionTreeRegressor(**study.best_trial.params)
        elif best_model_name == "Gradient Boosting":
            best_model = GradientBoostingRegressor(**study.best_trial.params)
        elif best_model_name == "Linear Regression":
            best_model = LinearRegression()
        elif best_model_name == "XGBRegressor":
            best_model = XGBRegressor(**study.best_trial.params)
        elif best_model_name == "CatBoosting Regressor":
            best_model = CatBoostRegressor(**study.best_trial.params, verbose=False)
        elif best_model_name == "AdaBoost Regressor":
            best_model = AdaBoostRegressor(**study.best_trial.params)

        # Train the best model on the entire training set
        best_model.fit(X_train, y_train)
        logging.info('Find best model completed')
        return best_model


In [92]:
file_path = r'D:\DS_project\ML_project\src\notebook\data\student.csv'

# Read the CSV file
df = pd.read_csv(file_path)
num_features = df.select_dtypes(exclude='object').columns
df_model = df.copy()
df_model['average_score'] = df[num_features].sum(axis=1)/len(num_features)
select_columns = ['math_score','reading_score','writing_score']
df_model.drop(select_columns,axis=1,inplace=True)
X = df_model.drop(columns=['average_score'],axis = 1)
y = df_model['average_score']
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(sparse=False)
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

# Fit and transform the data
X_encoded = preprocessor.fit_transform(X)

# Fit the OneHotEncoder
oh_transformer.fit(X[cat_features])

# Get column names for one-hot encoded features
encoded_column_names = oh_transformer.get_feature_names_out(cat_features)

# Combine column names of one-hot encoded and numerical features
final_column_names = list(encoded_column_names) + list(X.select_dtypes(exclude="object").columns)

# Convert the transformed array to a DataFrame with column names
X_encoded_df = pd.DataFrame(X_encoded, columns=final_column_names)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded_df,y,test_size=0.2,random_state=42)


d:\DS_project\ML_project\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
d:\DS_project\ML_project\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [93]:
find_model = ModelTrainer()
get_model = find_model.initiate_model_train(X_train, X_test, y_train, y_test)
print(get_model)


[I 2024-03-08 17:41:05,701] A new study created in memory with name: no-name-dbd0374b-cea0-471c-8277-685e149ab0bf


[I 2024-03-08 17:41:06,353] Trial 0 finished with value: 198.71163994593883 and parameters: {'model_name': 'Random Forest', 'rf_n_estimators': 78}. Best is trial 0 with value: 198.71163994593883.
[I 2024-03-08 17:41:06,368] Trial 1 finished with value: 153.96108778211806 and parameters: {'model_name': 'Linear Regression'}. Best is trial 1 with value: 153.96108778211806.
[I 2024-03-08 17:41:06,516] Trial 2 finished with value: 166.5269935091893 and parameters: {'model_name': 'Gradient Boosting', 'gb_learning_rate': 0.019743712509384153, 'gb_n_estimators': 49, 'gb_subsample': 0.886410972618038}. Best is trial 1 with value: 153.96108778211806.
[I 2024-03-08 17:41:07,755] Trial 3 finished with value: 160.61830029197125 and parameters: {'model_name': 'AdaBoost Regressor', 'ab_learning_rate': 0.01616286535755046, 'ab_n_estimators': 212}. Best is trial 1 with value: 153.96108778211806.
[I 2024-03-08 17:41:08,298] Trial 4 finished with value: 157.83056457049918 and parameters: {'model_name': '

LinearRegression()
